In [ ]:
# Import all the necessary packages and functions to run the model
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input,Dense,Concatenate, BatchNormalization
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import classification_report
from keras import backend as K
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
# Loading all the arrays
train_index = np.load('./Data/labels_done_train.npy')
test_index = np.load('./Data/labels_done_test.npy')

train_external = np.load('./Data/english_train_external.npy')
test_external = np.load('./Data/english_test_external.npy')

train_neural = np.load('./Data/english_train_neural.npy')
test_neural = np.load('./Data/english_test_neural.npy')

train_statistical = np.load('./Data/english_train_statistical.npy')
test_statistical = np.load('./Data/english_test_statistical.npy')

In [ ]:
stance_to_index = {'agree':0,'disagree':1,'discuss':2,'unrelated':3}
index_to_stance = {0:'agree',1:'disagree',2:'discuss',3:'unrelated'}

In [ ]:
df_train = pd.read_csv('./Data/FinalTrainSet.csv')
Y_temp = df_train[['Stance']]
Y_temp = Y_temp.values.tolist()
Y_train = [stance_to_index[Y_temp[i][0]] for i in train_index] 

In [ ]:
df_test = pd.read_csv('./Data/FinalTestSet.csv')
Y_temp1 = df_test[['Stance']]
Y_temp1 = Y_temp1.values.tolist()
Y_test = [stance_to_index[Y_temp1[i][0]] for i in test_index]
Y_check = Y_test

In [ ]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = np.zeros(categories,dtype=float)
        cats[s] = 1.0
        #for item in s:
            
            
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [ ]:
Y_train = to_categorical(Y_train, len(stance_to_index))

In [ ]:
# Defining the model paramaters. For more information, refer to the project report. 

inputA = Input(shape=(4800,))
inputB = Input(shape=(10000,))
inputC = Input(shape=(21,))
# the first branch operates on the first input
x = Dense(2048, activation="relu")(inputA)
x = Dense(1024,activation='relu')(x)
x = Dense(512,activation='relu')(x)
x = Dense(256, activation="relu")(x)
x = Dense(128,activation='relu')(x)
x_model = Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = Dense(512, activation="relu")(inputB)
y = Dense(256,activation='relu')(inputB)
y = Dense(128, activation="relu")(y)
y = Dense(64,activation='relu')(y)
y_model = Model(inputs=inputB, outputs=y)

z = Dense(4, activation = "relu")(inputC)
z_model = Model(inputs = inputC, outputs = z)
# combine the output of the three branches
combined = Concatenate()([x_model.output, z_model.output, y_model.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z1 = BatchNormalization()(combined)
#z = Dense(2, activation="relu")(combined)
z1 = Dense(4, activation="softmax")(z1)
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x_model.inputs, z_model.inputs,  y_model.inputs], outputs=z1)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(0.001),metrics=['accuracy'])


In [ ]:
model.fit(x = [train_neural,train_external,train_statistical], y = np.array(Y_train),batch_size=8,epochs=8,validation_split=0.2,shuffle=True)

In [ ]:
def ret_tags(sequences, index):
    tags = []
    for sequence in sequences:
        temp = index[np.argmax(sequence)]
        
        #temp.append(index[np.argmax(sequence)])
 
        tags.append(temp)
 
    return tags

In [ ]:
scores = model.evaluate([test_neural,test_external,test_statistical], to_categorical(Y_test, len(stance_to_index))) 

print(f"{model.metrics_names[1]}: {scores[1] * 100}")

In [ ]:
y_pred = model.predict(x = [train_neural,train_external,train_statistical])
predictions = ret_tags(y_pred, {i: t for t, i in stance_to_index.items()})

In [ ]:
Yres = [index_to_stance[x] for x in Y_check]

In [ ]:
# Classification Report
print(classification_report(Yres, predictions))

In [ ]:
cm = confusion_matrix(Yres,predictions)

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
# Plotting the confusion matrix

plot_confusion_matrix(cm,normalize=False,target_names = ['agree','disagree','discuss','unrelated'],title='Confusion Matrix for Stance Detection on the test data')

In [ ]:
# Saving the Model
model.save('./Data/FinalEngModel.h5')